# **Предобработка данных**

##### Задача
В группе компаний Тинькофф есть команда edTech, создающая платформу для обучающих курсов.
У команды edTech возникает вопрос, а какие курсы сильнее всего влияют на рабочие показатели сотрудников в колл-центре. 
Помогите составить рекомендации, какие обучающие курсы стоит проходить сотрудникам, а какие курсы стоит убрать с edTech платформы.
Решение может быть как рекомендательной моделью для каждого из сотрудников, так и основано на бизнес-правилах и статистическом анализе (например, выделить для каждого департамента полезные курсы).

##### Потенциальное решение
Предполагается, что решение будет выполнено на Python, будет итоговая презентация. Решение может содержать блоки: эксплораторный анализ, Causal Inference методы, рекомендательную модель.
Ограничений на подход к решению нет, но для определения важности курсов советуем использовать методы анализа причинности. Подробнее можно почитать о них [здесь,](https://koch-kir.medium.com/causal-inference-from-observational-data-%D0%B8%D0%BB%D0%B8-%D0%BA%D0%B0%D0%BA-%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D1%81%D1%82%D0%B8-%D0%B0-%D0%B2-%D1%82%D0%B5%D1%81%D1%82-%D0%B1%D0%B5%D0%B7-%D0%B0-%D0%B2-%D1%82%D0%B5%D1%81%D1%82%D0%B0-afb84f2579f2) а также возможна консультация со стороны организаторов.

##### Описание таблиц
###### employees
Информация о сотрудниках колл-центра
Поля:
- employee_id - идентификатор сотрудника
- sex – пол
- region - идентификатор федерального округа
- age – возраст
- head_employee_id – идентификатор руководителя
- exp_days – опыт в днях
- edu_degree – уровень образования
- department_id – индентификатор департамента, в котором работает сотрудник
- work_online_flg – флаг работы на удалённом режиме

###### communications
Информация о рабочих показателях сотрудников. Рассматривались рабочие коммункации операторов колл-центра
Поля:
- communication_id – идентификатор коммуникации
- communication_dt – дата коммуникации
- employee_id - идентификатор сотрудника
- communication_score – оценка качества коммуникации
- util_flg – флаг того, что клиент воспользовался банковским продуктом в течение 2 недель

###### courses_passing
Статиситка прохождения обучающих курсов сотрудниками
- course_id – идентификатор курса
- employee_id - идентификатор сотрудника
- pass_frac – доля прохождения курса
- start_dt – дата начала прохождения
- last_activity_dt – последняя активность сотрудника в обучающем курсе
- end_dt – дата окончания обучения. Если обучение пройдено не полностью, то NaN
- educ_duration_days – длительность полного обучения в днях. Если обучение пройдено не полностью, то NaN

###### courses_info 
Информация о курсах
- course_id – идентификатор курса
- course_nm – название курса

###### course_employee_sms 
Сводная таблица с нотификациями сотрудникам с предложением пройти обучение. Нотификации рассылались случайным образом
Поля:
- employee_id - идентификатор сотрудника
- course_i – флаг наличия нотификации

In [1]:
# импортируем библиотеки для обработки данных
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# импортируем другие библиотеки
import warnings

## **Откроем файлы с данными и изучим общую информацию**

In [2]:
warnings.filterwarnings('ignore')

In [12]:
# Установка максимального количества отображаемых строк
pd.set_option('display.max_rows', 20)

# Установка опции для отображения нормальных чисел
pd.set_option('display.float_format', lambda x: '%.2f' % x)

Попробуем открыть все датасеты

In [7]:
# Загрузка каждого датасета в отдельный датафрейм
communications_df = pd.read_csv('../data/src/communications.csv', sep=';')
courses_info_df = pd.read_csv('../data/src/courses_info.csv', sep=';')
courses_passing_df = pd.read_csv('../data/src/courses_passing.csv', sep=';')
course_employee_sms_df = pd.read_csv('../data/src/course_employee_sms.csv', sep=';')
employees_df = pd.read_csv('../data/src/employees.csv', sep=';')

### Рассмотрим датасет communications

In [5]:
# Расмотрим загруженные данные
communications_df.head()

,communication_id,communication_dt,employee_id,communication_score,util_flg
0,177074281189583840,2023-02-07 00:00:00,269d837a-fada-308d-d4ae-ab28ca2d57e4,85,0
1,189925315406757324,2023-07-29 00:00:00,e6384711-4917-13d2-9bc6-3fc5eeb5ba4f,100,0
2,321818238739717242,2023-09-06 00:00:00,ab541d87-4c7b-c19a-b776-42849e02b89f,43,1
3,192259329962028989,2023-06-19 00:00:00,0a0a0c8a-aa00-ade5-0f74-a3f0ca981ed7,100,1
4,705274671052151857,2023-08-11 00:00:00,f93882cb-d8fc-7fb7-94c1-011d63be6fb6,46,0


In [6]:
# Расмотрим информацию о данных
communications_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5345246 entries, 0 to 5345245
Data columns (total 5 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   communication_id     int64 
 1   communication_dt     object
 2   employee_id          object
 3   communication_score  int64 
 4   util_flg             int64 
dtypes: int64(3), object(2)
memory usage: 203.9+ MB


In [13]:
# Расмотрим основные характеристики
communications_df.describe(include='all')

,communication_id,communication_dt,employee_id,communication_score,util_flg
count,5345246.00,5345246,5345246,5345246.00,5345246.00
unique,NaN,354,2381,NaN,NaN
top,NaN,2023-12-16 00:00:00,a07c2f3b-3b90-7aaf-8436-a26c6d77f0a2,NaN,NaN
freq,NaN,22680,3214,NaN,NaN
mean,500199033041779136.00,NaN,NaN,58.56,0.75
std,291762749089854400.00,NaN,NaN,37.08,0.43
min,1052382138.00,NaN,NaN,0.00,0.00
25%,243720040218200704.00,NaN,NaN,29.00,1.00
50%,500446782549360256.00,NaN,NaN,67.00,1.00
75%,756578332801364992.00,NaN,NaN,100.00,1.00


**Основные метрики:**
- **Количество записей**: В датасете присутствует 5,345,246 записей о коммуникациях.
- **Количество уникальных дней**: В данных присутствуют коммуникации на протяжении 354 дней.
- **Количество сотрудников**: В данных упоминается 2 381 уникальный сотрудник.
- **Самая активная дата коммуникаций**: Наиболее активным днем для коммуникаций был `2023-12-16`, в течение которого было зарегистрировано 22 680 коммуникаций. Видимо сказалось предновогоднее время.
- **Самый активный сотрудник**: Сотрудник с идентификатором `a07c2f3b-3b90-7aaf-8436-a26c6d77f0a2` был наиболее активным, сделав 3 214 коммуникаций за этот период.

**Статистики оценок коммуникации (`communication_score`):**
- **Средняя оценка**: Средняя оценка коммуникации составила 58.56, что может свидетельствовать о достаточно высоком качестве коммуникаций в целом.
- **Стандартное отклонение**: Стандартное отклонение оценок равно 37.08, что указывает на значительную вариативность в оценках качества коммуникации.
- **Минимальная оценка**: Наименьшая зарегистрированная оценка составила 0.
- **Максимальная оценка**: Максимальная оценка достигла 100, что является пределом шкалы оценки.
- **Медиана**: Медианное значение оценок составило 67, что выше среднего, указывая на то, что более половины оценок находятся выше среднего уровня.
- **Первый квартиль**: 25% оценок находятся ниже 29, что говорит о наличии значительного количества низких оценок.
- **Третий квартиль**: 75% оценок ниже 100, что свидетельствует о широком размахе оценок, но при этом 25% составляет оценку 100.

**Использование продукта (`util_flg`):**
- **Средний показатель использования**: В среднем, 75% коммуникаций заканчиваются тем, что клиенты воспользовались продуктом в течение 2 недель после коммуникации, что свидетельствует о высокой эффективности коммуникаций.
- **Стандартное отклонение**: Стандартное отклонение равно 0.43, что указывает на вариативность в успешности коммуникаций между различными сотрудниками или в разные периоды.
- **Минимальное и максимальное значения**: Минимальное значение равно 0 (клиент не воспользовался продуктом), а максимальное – 1 (клиент воспользовался продуктом), что подтверждает бинарную природу этого показателя.

### Рассмотрим датасет courses_info

In [14]:
# Расмотрим загруженные данные
courses_info_df.head()

,course_id,course_nm
0,3,Улучшение качества обслуживания клиентов: Осно...
1,76,Как эффективно использовать психологию в обслу...
2,22,Эффективное обучение клиентов в использовании ...
3,53,Как эффективно реагировать на изменения потреб...
4,7,Построение долгосрочных отношений с клиентами:...


In [15]:
# Расмотрим информацию о данных
courses_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   course_id  92 non-null     int64 
 1   course_nm  92 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.6+ KB


In [16]:
# Расмотрим основные характеристики
courses_info_df.describe(include='all')

,course_id,course_nm
count,92.00,92
unique,NaN,92
top,NaN,Улучшение качества обслуживания клиентов: Осно...
freq,NaN,1
mean,45.50,NaN
std,26.70,NaN
min,0.00,NaN
25%,22.75,NaN
50%,45.50,NaN
75%,68.25,NaN


**Информация о курсах (`course_nm`):**
- **Количество курсов**: Всего курсов - 92.

### Рассмотрим датасет courses_passing

In [20]:
# Расмотрим загруженные данные
courses_passing_df.head()

,course_id,employee_id,pass_frac,start_dt,last_activity_dt,end_dt,educ_duration_days
0,24,53ed35c7-4a2e-c275-b837-374f04396c03,1.00,2023-01-30 00:00:00,2023-02-05 00:00:00,2023-02-05 00:00:00,6.00
1,83,202cb962-ac59-075b-964b-07152d234b70,1.00,2023-08-29 00:00:00,2023-09-10 00:00:00,2023-09-10 00:00:00,12.00
2,4,05f971b5-ec19-6b8c-65b7-5d2ef8267331,1.00,2023-07-29 00:00:00,2023-08-09 00:00:00,2023-08-09 00:00:00,11.00
3,79,f4a331b7-a22d-1b23-7565-d8813a34d8ac,1.00,2023-05-30 00:00:00,2023-06-09 00:00:00,2023-06-09 00:00:00,10.00
4,6,aba3b6fd-5d18-6d28-e06f-f97135cade7f,1.00,2023-07-12 00:00:00,2023-07-22 00:00:00,2023-07-22 00:00:00,10.00


In [21]:
# Расмотрим информацию о данных
courses_passing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10950 entries, 0 to 10949
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   course_id           10950 non-null  int64  
 1   employee_id         10950 non-null  object 
 2   pass_frac           10950 non-null  float64
 3   start_dt            10950 non-null  object 
 4   last_activity_dt    10950 non-null  object 
 5   end_dt              8404 non-null   object 
 6   educ_duration_days  8404 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 599.0+ KB


In [22]:
# Расмотрим основные характеристики
courses_passing_df.describe(include='all')

,course_id,employee_id,pass_frac,start_dt,last_activity_dt,end_dt,educ_duration_days
count,10950.00,10950,10950.00,10950,10950,8404,8404.00
unique,NaN,1758,NaN,295,308,307,NaN
top,NaN,8f1d4362-0bc6-bb58-0df6-e80b0dc05c48,NaN,2023-01-11 00:00:00,2023-05-24 00:00:00,2023-05-24 00:00:00,NaN
freq,NaN,32,NaN,136,78,66,NaN
mean,44.32,NaN,0.88,NaN,NaN,NaN,10.04
std,26.90,NaN,0.26,NaN,NaN,NaN,3.12
min,0.00,NaN,0.00,NaN,NaN,NaN,1.00
25%,20.00,NaN,1.00,NaN,NaN,NaN,8.00
50%,43.00,NaN,1.00,NaN,NaN,NaN,10.00
75%,67.75,NaN,1.00,NaN,NaN,NaN,12.00


**Обзор данных по обучению сотрудников:**

- **Количество записей**: 10 950 записей, что указывает на активное участие сотрудников в курсах.
- **Уникальные сотрудники**: Данные охватывают 1 758 уникальных сотрудников, многие из которых, возможно, прошли по несколько курсов.
- **Самый активный сотрудник**: Сотрудник с идентификатором `8f1d4362-0bc6-bb58-0df6-e80b0dc05c48` участвовал в наибольшем количестве курсов (32 раза).

**Прохождение курсов (`pass_frac`):**
- **Средний процент успешного завершения курсов**: 0.88, что свидетельствует о высоком уровне "успеваемости" среди сотрудников.
- **Стандартное отклонение**: 0.26, показывая разнообразие в "успеваемости" среди разных сотрудников или курсов.

**Продолжительность обучения (`educ_duration_days`):**
- **Средняя продолжительность обучения**: 10.04 дня, что указывает на относительно короткие курсы.
- **Стандартное отклонение продолжительности обучения**: 3.12 дней, показывая вариативность в длительности различных курсов.
- **Минимальная продолжительность обучения**: 1 день, что может указывать на интенсивные краткосрочные курсы или, к примеруу, оценочные тесты.
- **Максимальная продолжительность обучения**: 23 дня, что может соответствовать более глубоким и подробным курсам.

**Анализ:**
- Данные показывают активное участие сотрудников в обучающих программах, с высоким процентом успешного завершения курсов.
- Вариативность в продолжительности курсов может указывать на широкий спектр предлагаемых обучающих программ, от коротких интенсивных курсов до более длительных и глубоких обучений.

### Рассмотрим датасет course_employee_sms

In [23]:
# Расмотрим загруженные данные
course_employee_sms_df.head()

,employee_id,course_0,course_1,course_2,course_3,course_4,course_5,course_6,course_7,course_8,...,course_82,course_83,course_84,course_85,course_86,course_87,course_88,course_89,course_90,course_91
0,2a38a4a9-316c-49e5-a833-517c45d31070,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,4e0928de-0755-38c5-93fb-dabb0c5ef2c3,0,0,0,1,0,0,0,0,1,...,1,1,1,1,0,0,0,0,1,0
2,1ff1de77-4005-f8da-13f4-2943881c655f,1,0,1,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,0
3,cd3afef9-b8b8-9558-cd56-638c3631868a,0,0,1,1,1,0,0,0,1,...,1,0,1,1,1,0,1,1,1,0
4,dc513ea4-fbda-a7a1-4786-ffdebc4ef64e,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [24]:
# Расмотрим информацию о данных
course_employee_sms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381 entries, 0 to 2380
Data columns (total 93 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   employee_id  2381 non-null   object
 1   course_0     2381 non-null   int64 
 2   course_1     2381 non-null   int64 
 3   course_2     2381 non-null   int64 
 4   course_3     2381 non-null   int64 
 5   course_4     2381 non-null   int64 
 6   course_5     2381 non-null   int64 
 7   course_6     2381 non-null   int64 
 8   course_7     2381 non-null   int64 
 9   course_8     2381 non-null   int64 
 10  course_9     2381 non-null   int64 
 11  course_10    2381 non-null   int64 
 12  course_11    2381 non-null   int64 
 13  course_12    2381 non-null   int64 
 14  course_13    2381 non-null   int64 
 15  course_14    2381 non-null   int64 
 16  course_15    2381 non-null   int64 
 17  course_16    2381 non-null   int64 
 18  course_17    2381 non-null   int64 
 19  course_18    2381 non-null 

In [25]:
# Расмотрим основные характеристики
course_employee_sms_df.describe(include='all')

,employee_id,course_0,course_1,course_2,course_3,course_4,course_5,course_6,course_7,course_8,...,course_82,course_83,course_84,course_85,course_86,course_87,course_88,course_89,course_90,course_91
count,2381,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00,...,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00
unique,2381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2a38a4a9-316c-49e5-a833-517c45d31070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.30,0.30,0.31,0.30,0.31,0.31,0.30,0.31,0.30,...,0.29,0.28,0.30,0.30,0.28,0.30,0.32,0.30,0.30,0.32
std,NaN,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.46,...,0.45,0.45,0.46,0.46,0.45,0.46,0.47,0.46,0.46,0.47
min,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,NaN,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


**Обзор данных по курсам и участию сотрудников:**

- **Количество записей**: Данные включают информацию о 2 381 сотрудниках. Все 2 381 записи относятся к уникальным сотрудникам, каждый идентифицируется своим `employee_id`.

**Участие в курсах:**

- Каждый сотрудник имеет данные об участии в 92 различных курсах (`course_0` до `course_91`).
- **Среднее участие**: Для большинства курсов среднее участие (прохождение курса) колеблется около 0.30, что указывает на то, что в среднем около 30% сотрудников прошли каждый курс.
- **Стандартное отклонение**: Стандартное отклонение участия в курсах варьируется около 0.46, что говорит о значительном разбросе в данных: многие сотрудники не прошли курс (0), в то время как другие успешно его завершили (1).

**Анализ:**

- **Активность сотрудников**: Данные показывают, что сотрудники вовлечены в различные образовательные программы, однако степень участия сильно различается между отдельными курсами и сотрудниками.

### Рассмотрим датасет employees

In [26]:
# Расмотрим загруженные данные
employees_df.head()

,employee_id,sex,region,age,head_employee_id,exp_days,edu_degree,department_id,work_online_flg
0,0f840be9-b8db-4d3f-bd5b-a2ce59211f55,M,4,36,c81e728d-9d4c-2f63-6f06-7f89cc14862c,734,0,2,1
1,ac1dd209-cbcc-5e5d-1c6e-28598e8cbbe8,M,4,32,93db85ed-909c-1383-8ff9-5ccfa94cebd9,654,0,2,1
2,a8c88a00-55f6-36e4-a163-a5e3d16adab7,F,5,34,9a115815-4dfa-42ca-ddbd-0694a4e9bdc8,276,1,2,1
3,c91591a8-d461-c286-9b9f-535ded3e213e,F,4,33,9f61408e-3afb-633e-50cd-f1b20de6f466,50,2,0,1
4,bc731692-9fe1-545b-f0b9-8d114ee3ecb8,M,0,33,a97da629-b098-b75c-294d-ffdc3e463904,274,1,2,1


In [27]:
# Расмотрим информацию о данных
employees_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381 entries, 0 to 2380
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   employee_id       2381 non-null   object
 1   sex               2381 non-null   object
 2   region            2381 non-null   int64 
 3   age               2381 non-null   int64 
 4   head_employee_id  2381 non-null   object
 5   exp_days          2381 non-null   int64 
 6   edu_degree        2381 non-null   int64 
 7   department_id     2381 non-null   int64 
 8   work_online_flg   2381 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 167.5+ KB


In [28]:
# Расмотрим основные характеристики
employees_df.describe(include='all')

,employee_id,sex,region,age,head_employee_id,exp_days,edu_degree,department_id,work_online_flg
count,2381,2381,2381.00,2381.00,2381,2381.00,2381.00,2381.00,2381.00
unique,2381,2,NaN,NaN,132,NaN,NaN,NaN,NaN
top,0f840be9-b8db-4d3f-bd5b-a2ce59211f55,M,NaN,NaN,3c59dc04-8e88-5024-3be8-079a5c74d079,NaN,NaN,NaN,NaN
freq,1,1252,NaN,NaN,28,NaN,NaN,NaN,NaN
mean,NaN,NaN,2.77,32.65,NaN,461.73,1.55,1.43,0.57
std,NaN,NaN,2.29,6.75,NaN,374.56,0.60,0.70,0.50
min,NaN,NaN,0.00,19.00,NaN,0.00,0.00,0.00,0.00
25%,NaN,NaN,0.00,28.00,NaN,180.00,1.00,1.00,0.00
50%,NaN,NaN,3.00,32.00,NaN,362.00,2.00,2.00,1.00
75%,NaN,NaN,4.00,37.00,NaN,638.00,2.00,2.00,1.00


**Профили сотрудников:**

- **Количество записей**: Данные охватывают 2 381 сотрудника.

**Демографические и профессиональные характеристики:**

- **Пол**: Среди сотрудников 1 252 мужчин и остальные женщины, что делает распределение полов относительно сбалансированным.
- **Регион**: Сотрудники распределены по 8 регионам (от 0 до 7).
- **Возраст**: Средний возраст сотрудников составляет 32.65 лет со стандартным отклонением 6.75 лет, диапазон возрастов - от 19 до 60 лет, что свидетельствует о широком возрастном разнообразии в коллективе.
- **Опыт работы (в днях)**: Средний стаж работы сотрудников составляет 461.73 дня, с максимальным стажем в 2424 дня, что указывает на наличие как новичков, так и опытных сотрудников.
- **Уровень образования**: Значения `edu_degree` варьируются от 0 до 2, что может указывать на различные уровни образовательной подготовки сотрудников.
- **Работа онлайн**: Приблизительно 57% сотрудников имеют возможность работать онлайн (`work_online_flg` = 1), что подчеркивает гибкость рабочих условий.

**Анализ:**

- **Разнообразие в коллективе**: Данные подчеркивают разнообразие сотрудников по полу, возрасту, региону проживания и образовательному уровню, что может способствовать широкому спектру идей и подходов в рабочем процессе.
- **Гибкость в работе**: Наличие возможности работать онлайн у значительной части сотрудников говорит о гибкости в организации рабочего процесса, что может быть особенно актуально в современных условиях.
- **Опыт работы**: Распределение по стажу работы показывает, что в коллективе присутствуют как новые, так и опытные кадры, что может способствовать обмену знаниями и опытом внутри организации.

## **Предобработка данных**
### **Обработка дубликатов**

Проверим датасеты на наличие дубликатов

In [30]:
# Список всех DataFrame'ов
dataframes = [communications_df, 
              courses_info_df,
              courses_passing_df,
              course_employee_sms_df,
              employees_df]

# Названия таблиц для удобства вывода
table_names = ['communications', 
               'courses_info', 
               'courses_passing', 
               'course_employee_sms', 
               'employees']  

for df, name in zip(dataframes, table_names):
    # Проверка на наличие дубликатов
    duplicates = df.duplicated().sum()

    if duplicates > 0:
        print(f'В {name} найдено {duplicates} дубликатов.')
    else:
        print(f'В {name} дубликатов не обнаружено.')

В communications дубликатов не обнаружено.
В courses_info дубликатов не обнаружено.
В courses_passing дубликатов не обнаружено.
В course_employee_sms дубликатов не обнаружено.
В employees дубликатов не обнаружено.


### **Обработка пропусков**

Проверим датасеты на наличие пропусков

In [31]:
for df, name in zip(dataframes, table_names):
    # Проверка на наличие пропусков
    missing_values = df.isnull().sum().sum()

    if missing_values > 0:
        print(f'В {name} найдено {missing_values} пропусков.')
    else:
        print(f'В {name} пропусков не обнаружено.')

В communications пропусков не обнаружено.
В courses_info пропусков не обнаружено.
В courses_passing найдено 5092 пропусков.
В course_employee_sms пропусков не обнаружено.
В employees пропусков не обнаружено.


In [33]:
# еще раз рассмотрим информацию по датасету courses_passing_df
courses_passing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10950 entries, 0 to 10949
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   course_id           10950 non-null  int64  
 1   employee_id         10950 non-null  object 
 2   pass_frac           10950 non-null  float64
 3   start_dt            10950 non-null  object 
 4   last_activity_dt    10950 non-null  object 
 5   end_dt              8404 non-null   object 
 6   educ_duration_days  8404 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 599.0+ KB


Данные пропуски свидетельствуют, что на момент создания датасета часть сотрудников не закончила курсы.
Также стоит отметить, что во всех датасетах отсутствуют аномальные значения.

## Объединение таблиц

In [34]:
# Объединение Профилей сотрудников и Участия в курсах
merged_df = pd.merge(courses_passing_df, employees_df, on='employee_id', how='left')

In [37]:
# Добавление информации о курсах из Courses Info к уже объединенному DataFrame
merged_df = pd.merge(merged_df, courses_info_df, on='course_id', how='left')

In [38]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10950 entries, 0 to 10949
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   course_id           10950 non-null  int64  
 1   employee_id         10950 non-null  object 
 2   pass_frac           10950 non-null  float64
 3   start_dt            10950 non-null  object 
 4   last_activity_dt    10950 non-null  object 
 5   end_dt              8404 non-null   object 
 6   educ_duration_days  8404 non-null   float64
 7   sex                 10950 non-null  object 
 8   region              10950 non-null  int64  
 9   age                 10950 non-null  int64  
 10  head_employee_id    10950 non-null  object 
 11  exp_days            10950 non-null  int64  
 12  edu_degree          10950 non-null  int64  
 13  department_id       10950 non-null  int64  
 14  work_online_flg     10950 non-null  int64  
 15  course_nm           10950 non-null  object 
dtypes: f

In [39]:
# Сохраним датасет в файл
merged_df.to_csv('../data/preprocessed/courses_passing__employee.csv')